In [7]:
%pip install --upgrade openai python-dotenv azure-identity

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os, time, json, pathlib, sys, logging
from dotenv import load_dotenv

# ---------- Configure logger (avoid duplicate handlers in notebooks) ----------
logger = logging.getLogger("azure_openai_demo")
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(logging.Formatter("[%(levelname)s] %(message)s"))
    logger.addHandler(handler)
logger.setLevel(logging.INFO)

# ---------- Load .env (search upwards) ----------
cwd = pathlib.Path.cwd()
env_path = None
for parent in [cwd] + list(cwd.parents):
    candidate = parent / ".env"
    if candidate.is_file():
        env_path = candidate
        break

if env_path is None:
    logger.warning("No .env file found – you must set env vars manually.")
else:
    logger.info("Loading environment from %s", env_path)
    load_dotenv(env_path)

[INFO] Loading environment from /home/superdev/projects/OpenMates/.env


In [5]:
# --- Streamed Azure OpenAI chat completion ---
# This cell switches from a single, blocking response to streaming token-by-token output.
# - We write tokens to stdout for smooth, real-time display in notebooks.
# - We also use the configured logger (INFO level) for lifecycle messages and summaries.

import os
import sys
import base64  # kept in case you uncomment the image example later
from openai import AzureOpenAI

# --- Configuration (env with sensible defaults for local testing) ---
endpoint = os.getenv(
    "SECRET__OPENAI__AZURE__ENDPOINT_URL",
    "https://micro-me2ldtrn-eastus2.cognitiveservices.azure.com/",
)
deployment = os.getenv("SECRET__OPENAI__AZURE__DEPLOYMENT_NAME", "gpt-5")
subscription_key = os.getenv(
    "SECRET__OPENAI__AZURE__API_KEY", "REPLACE_WITH_YOUR_KEY_VALUE_HERE"
)

# --- Ensure a logger exists in this cell (in case cells run out-of-order) ---
try:
    logger  # type: ignore  # noqa: F821
except NameError:
    import logging

    logger = logging.getLogger("azure_openai_demo")
    if not logger.handlers:
        handler = logging.StreamHandler(sys.stdout)
        handler.setFormatter(logging.Formatter("[%(levelname)s] %(message)s"))
        logger.addHandler(handler)
    logger.setLevel(logging.INFO)

# --- Initialize Azure OpenAI client (key-based auth) ---
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("ascii")

# --- Prepare the chat prompt ---
chat_prompt = [
    {
        "role": "developer",
        "content": [{"type": "text", "text": "You are a helpful assistant."}],
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": (
                    "What is the capital of France?"
                    # "how do I integrate Oxylabs proxy into my python code? "
                    # "and can I integrate it also with the youtube_transcript_api pip package?"
                ),
            }
        ],
    },
]

# Include speech result if speech is enabled
messages = chat_prompt

# --- Stream the completion token-by-token ---
logger.info("Starting streamed chat.completions response (model=%s)", deployment)

accumulated_text_parts = []
try:
    stream = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_completion_tokens=16384,
        stop=None,
        stream=True,
        timeout=120.0,
    )

    # Write tokens directly to stdout for real-time display in the notebook.
    # We avoid 'print' and instead write raw tokens to stdout, while using the
    # logger for high-level lifecycle messages and the final summary below.
    for chunk in stream:
        # Defensive checks because some SDK versions may emit non-delta or empty chunks
        if not chunk or not getattr(chunk, "choices", None):
            continue

        delta = chunk.choices[0].delta
        if delta is None:
            continue

        content_piece = getattr(delta, "content", None)
        if content_piece:
            sys.stdout.write(content_piece)
            sys.stdout.flush()
            accumulated_text_parts.append(content_piece)
except Exception as exc:
    logger.exception("Error while streaming response: %s", exc)
finally:
    # Ensure a newline after streaming so the next cell's output starts cleanly
    sys.stdout.write("\n")
    sys.stdout.flush()

    final_text = "".join(accumulated_text_parts)
    logger.info("Stream finished. Characters received: %d", len(final_text))
    

[INFO] Starting streamed chat.completions response (model=gpt-5)


Paris.
[INFO] Stream finished. Characters received: 6
